In [4]:
import pandas as pd

In [5]:
# if you want to import the identify_missing_dat function

import os, sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src.helpers import identify_missing_data

In [25]:
# read the pickle file

users_data = pd.read_pickle("../data/users_data_final.pkl")

In [9]:
users_data.shape

(45209, 19)

In [17]:
users_data.isnull().sum()

user_id                    0
number_transactions     9999
total_amount_usd        9999
age                        0
job                      288
marital                    0
education               1857
default                    0
housing                    0
loan                       0
contact                13018
duration                   0
campaign                   0
pdays                      0
previous                   0
poutcome               36957
term_deposit               0
date_joined               30
device                    94
dtype: int64

In [10]:
# transformation task # 1, drop all columns where more than 70% missing data

users_data_transformed = users_data.drop(["poutcome"], axis=1)

In [11]:
users_data_transformed.shape

(45209, 18)

In [15]:
# transformation task # 2, drop all rows where date joined is missing
users_data_transformed = users_data_transformed[~users_data_transformed.date_joined.isnull()]

In [31]:
# option 2 - use the dropna function to remove rows where date joined has missing values
users_data_transformed.dropna(subset=['date_joined'], inplace=True)

users_data_transformed = users_data_transformed.dropna(subset=['date_joined'])

In [32]:
users_data_transformed.shape

(45179, 18)

In [19]:
# final step - load the data frame to the data folder
users_data_transformed.to_pickle("../data/users_data_transformed.pkl")

In [21]:
# read your loaded file that lives in YOUR data folder

check = pd.read_pickle("../data/users_data_transformed.pkl")
check.head()

,user_id,number_transactions,total_amount_usd,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,term_deposit,date_joined,device
0,9231c446-cb16-4b2b-a7f7-ddfc8b25aaf6,3.0,2143.0,58,management,married,tertiary,False,True,False,None,261,1,-1,0,False,1998-08-23,mobile
1,bb92765a-08de-4963-b432-496524b39157,NaN,NaN,44,technician,single,secondary,False,True,False,None,151,1,-1,0,False,2008-07-15,desktop
2,573de577-49ef-42b9-83da-d3cfb817b5c1,2.0,2.0,33,entrepreneur,married,secondary,False,True,True,None,76,1,-1,0,False,2002-06-04,mobile
3,d6b66b9d-7c8f-4257-a682-e136f640b7e3,NaN,NaN,47,blue-collar,married,None,False,True,False,None,92,1,-1,0,False,1995-06-29,tablet
4,fade0b20-7594-4d9a-84cd-c02f79b1b526,1.0,1.0,33,None,single,None,False,False,False,None,198,1,-1,0,False,1995-08-01,mobile


In [22]:
check.shape

(45179, 18)

In [50]:
# connect to s3
# create three new variables to connect to s3
# fill in the gaps yourselves

AWS_ACCESS_KEY_ID = ""
AWS_SECRET_ACCESS_KEY = ""
AWS_S3_BUCKET = "sep-bootcamp"


In [35]:
AWS_S3_BUCKET

'sep-bootcamp'

In [51]:
# make a connection to s3
import boto3

s3_client = boto3.client(
    "s3",
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY
)
s3_client

In [45]:
# if you need to install boto3 - this is a one off task
#! pip3 install boto3

In [54]:
# load the data frame to s3 bucket
from io import StringIO
key = "transformations/sh_user_data_final.csv"

csv_buffer = StringIO() # create buffer to temporarily store the Data Frame
users_data_transformed.to_csv(csv_buffer, index=False) # code to write the data frame as csv file
response = s3_client.put_object(
        Bucket=AWS_S3_BUCKET, Key=key, Body=csv_buffer.getvalue()
    )

In [56]:
# if you want to read your file from the s3 bucket
# or read someone elses

key = "transformations/sa_user_data_final.csv" # update the key

response = s3_client.get_object(Bucket=AWS_S3_BUCKET, Key=key)
santhi_file = pd.read_csv(response.get("Body"))

In [58]:
santhi_file.shape

(45209, 18)

In [61]:
key = "transformations/nn_user_data_final.csv" # try natalias

response = s3_client.get_object(Bucket=AWS_S3_BUCKET, Key=key)
natalia_file = pd.read_csv(response.get("Body"))

In [62]:
natalia_file.shape

(45179, 18)

In [63]:
natalia_file.head()

,user_id,number_transactions,total_amount_usd,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,term_deposit,date_joined,device
0,9231c446-cb16-4b2b-a7f7-ddfc8b25aaf6,3.0,2143.0,58,management,married,tertiary,False,True,False,NaN,261,1,-1,0,False,1998-08-23,mobile
1,bb92765a-08de-4963-b432-496524b39157,NaN,NaN,44,technician,single,secondary,False,True,False,NaN,151,1,-1,0,False,2008-07-15,desktop
2,573de577-49ef-42b9-83da-d3cfb817b5c1,2.0,2.0,33,entrepreneur,married,secondary,False,True,True,NaN,76,1,-1,0,False,2002-06-04,mobile
3,d6b66b9d-7c8f-4257-a682-e136f640b7e3,NaN,NaN,47,blue-collar,married,NaN,False,True,False,NaN,92,1,-1,0,False,1995-06-29,tablet
4,fade0b20-7594-4d9a-84cd-c02f79b1b526,1.0,1.0,33,NaN,single,NaN,False,False,False,NaN,198,1,-1,0,False,1995-08-01,mobile
